In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import radians, sin, cos, sqrt, atan2
import os
import tarfile
import gzip
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import jaccard_score
from sklearn.metrics import mean_squared_error

In [4]:
def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = 6371 * c  # Radius of the Earth in kilometers
    return distance

# ? Function to extract data from the tower tar file
# def extractData(tower_tar_file_path):
#     tower_data = pd.DataFrame(columns=["Timestamp", "Year", "JulianDay", "Time", "BatteryVoltage",
#                                        "PanelTemperature", "BarometricPressure", "Rainfall",
#                                        "Temperature_2m", "RelativeHumidity_2m", "Temperature_10m",
#                                        "SolarRadiation_W/m2", "SolarRadiation_kW/m2", "Date"])
#     all_tower_data = pd.DataFrame(columns=tower_data.columns)
#     tar_file = tarfile.open(tower_tar_file_path, 'r:gz')
#     tar_file.extractall(os.path.dirname(tower_tar_file_path))
#     fileNames = []
#     for member in tar_file.getmembers():
#         if member.isfile() and member.name.endswith('logger.txt.gz'):
#             fileName = member.name.split("/")[2]
#             if (fileName.startswith(("20181101", "20181102", "20181103", "20181104", "20181105"))):
#                 with tar_file.extractfile(member) as f:
#                     date = os.path.basename(os.path.dirname(member.name))
#                     with gzip.open(f, 'rt') as g:
#                         tower_data = pd.read_csv(g, header=None, names=tower_data.columns, skiprows=1)
#                         tower_data['Date'] = date
#                         all_tower_data = pd.concat([all_tower_data, tower_data], ignore_index=True)
#                 fileNames.append(member.name)
#     tar_file.close()
#     print("File Names: ", fileNames)
#     print("Number of Files: ", len(fileNames))
#     return all_tower_data

def extractData(tower_tar_file_path):
    tower_data = pd.DataFrame(columns=["Timestamp", "Year", "JulianDay", "Time", "BatteryVoltage",
                                       "PanelTemperature", "BarometricPressure", "Rainfall",
                                       "Temperature_2m", "RelativeHumidity_2m", "Temperature_10m",
                                       "SolarRadiation_W/m2", "SolarRadiation_kW/m2", "Date"])
    all_tower_data = pd.DataFrame(columns=tower_data.columns)

    # Open the tar file
    with tarfile.open(tower_tar_file_path, 'r:gz') as tar_file:
        fileNames = []

        # Iterate through members in the tar file
        for member in tar_file.getmembers():
            if member.isfile() and member.name.endswith('logger.txt.gz'):
                fileName = member.name.split("/")[2]
                date_strings = [f"201811{str(i).zfill(2)}" for i in range(1, 31)]
                if fileName.startswith(tuple(date_strings)):
                    with tar_file.extractfile(member) as f:
                        date = os.path.basename(os.path.dirname(member.name))

                        # Read the contents of the gzip file directly into a DataFrame
                        with gzip.open(f, 'rt') as g:
                            tower_data = pd.read_csv(g, header=None, names=tower_data.columns, skiprows=1,on_bad_lines='skip')
                            tower_data['Date'] = date
                            all_tower_data = pd.concat([all_tower_data, tower_data], ignore_index=True)
                    fileNames.append(member.name)

        print("File Names: ", fileNames)
        print("Number of Files: ", len(fileNames))

    return all_tower_data

# ? Function to preprocess the data
def preprocess_data(data):
    # Replace "NAN" with NaN
    data.replace("NAN", float("NaN"), inplace=True)

    # Check for null values
    null_values = data.isnull().sum()
    print("Null values:\n", null_values)

    # Check for infinite values
    inf_values = data.isin([float('inf'), float('-inf')]).sum()
    print("\nInfinite values:\n", inf_values)

    # Remove rows with null or infinite values
    data = data.dropna()
    data = data[~data.isin([float('inf'), float('-inf')]).any(axis=1)]

    return data

# ? Function to group data by date
def group_data_by_date(data):
    # Convert 'Time' column to HH:MM format
    data['Time'] = data['Time'].apply(lambda x: '{:04}'.format(x))
    data['Time'] = data['Time'].str[:2] + ':' + data['Time'].str[2:]

    # Remove extra colons from 'Time' column
    data['Time'] = data['Time'].str.replace(':', '')

    # Convert 'Date' column to string type
    data['Date'] = data['Date'].astype(str)

    # Combine 'Date' and 'Time' columns to create a new datetime column
    data['DateTime'] = pd.to_datetime(data['Date'] + ' ' + data['Time'], format='%Y%m%d %H%M')

    # Group by 'Date' and aggregate using mean for specified columns
    aggregated_data = data.groupby('Date').agg({
        'BatteryVoltage': 'mean',
        'PanelTemperature': 'mean',
        'BarometricPressure': 'mean',
        'Rainfall': 'mean',
        'Temperature_2m': 'mean',
        'RelativeHumidity_2m': 'mean',
        'Temperature_10m': 'mean',
        'SolarRadiation_W/m2': 'mean',
        'SolarRadiation_kW/m2': 'mean'
    }).reset_index()

    return aggregated_data

# ? Function to group data by hourly period
def group_data_hourly(data):
    # Convert 'Time' column to HH:MM format
    data['Time'] = data['Time'].apply(lambda x: '{:04}'.format(x))
    data['Time'] = data['Time'].str[:2] + ':' + data['Time'].str[2:]

    # Remove extra colons from 'Time' column
    data['Time'] = data['Time'].str.replace(':', '')

    # Convert 'Date' column to string type
    data['Date'] = data['Date'].astype(str)

    # Combine 'Date' and 'Time' columns to create a new datetime column
    data['DateTime'] = pd.to_datetime(data['Date'] + ' ' + data['Time'], format='%Y%m%d %H%M')

    # Round datetime to the nearest hour
    data['HourlyDateTime'] = data['DateTime'].dt.round('H')

    # Group by hourly datetime and aggregate using mean for specified columns
    hourly_aggregated_data = data.groupby('HourlyDateTime').agg({
        'BatteryVoltage': 'mean',
        'PanelTemperature': 'mean',
        'BarometricPressure': 'mean',
        'Rainfall': 'mean',
        'Temperature_2m': 'mean',
        'RelativeHumidity_2m': 'mean',
        'Temperature_10m': 'mean',
        'SolarRadiation_W/m2': 'mean',
        'SolarRadiation_kW/m2': 'mean'
    }).reset_index()

    return hourly_aggregated_data

# ? Function to group data by half-hourly period
def group_data_half_hourly(processed_data):
    # Convert 'Time' column to HH:MM:SS format
    processed_data['Time'] = processed_data['Time'].apply(lambda x: '{:04}'.format(x))
    processed_data['Time'] = processed_data['Time'].str[:2] + ':' + processed_data['Time'].str[2:] + ':00'

    # Combine 'Date' and 'Time' columns to create a new datetime column
    processed_data['DateTime'] = pd.to_datetime(processed_data['Date'].astype(str) + ' ' + processed_data['Time'], format='%Y%m%d %H:%M:%S')

    # Calculate seconds after initial hour
    seconds_after_hour = processed_data['Timestamp'].astype(float)
    seconds_after_hour = (seconds_after_hour / 60).astype(int)  # Convert to seconds and round down
    processed_data['SecondsAfterHour'] = seconds_after_hour

    # Calculate exact datetime by adding seconds after hour to datetime
    processed_data['DateTime'] += pd.to_timedelta(processed_data['SecondsAfterHour'], unit='s')

    # Group by date and half-hourly period and aggregate using mean for specified columns
    half_hourly_aggregated_data = processed_data.groupby(['Date', pd.Grouper(key='DateTime', freq='30min')]).agg({
        'BatteryVoltage': 'mean',
        'PanelTemperature': 'mean',
        'BarometricPressure': 'mean',
        'Rainfall': 'mean',
        'Temperature_2m': 'mean',
        'RelativeHumidity_2m': 'mean',
        'Temperature_10m': 'mean',
        'SolarRadiation_W/m2': 'mean',
        'SolarRadiation_kW/m2': 'mean'
    }).reset_index()

    return half_hourly_aggregated_data

# ? Function to compare the change in temperature over time
def hourly_plot_comparison(towerA_data, towerB_data, metric, labelA, labelB):
    plt.figure(figsize=(12, 6))

    # Plot Temperature_2m for Tower A
    plt.plot(towerA_data['HourlyDateTime'], towerA_data[metric], label=labelA, color='blue')

    # Plot Temperature_2m for Tower B
    plt.plot(towerB_data['HourlyDateTime'], towerB_data[metric], label=labelB, color='red')

    plt.title('Hourly ' + metric + ' Comparison')
    plt.xlabel('Date')
    plt.ylabel(metric)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def half_hourly_plot_comparison(towerA_half_hourly, towerB_half_hourly, metric, labelA, labelB):
    plt.figure(figsize=(12, 6))

    # Plot Temperature_2m for Tower 8
    plt.plot(towerA_half_hourly['DateTime'], towerA_half_hourly[metric], label=labelA, color='green')

    # Plot Temperature_2m for Tower 11
    plt.plot(towerB_half_hourly['DateTime'], towerB_half_hourly['Temperature_2m'], label=labelB, color='orange')

    plt.title('Half-Hourly '+ metric +' Comparison')
    plt.xlabel('Date')
    plt.ylabel(metric)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


# ? Function to calculate similarity between two towers
def calculate_similarity(towerA_data, towerB_data, sensor_metric, day, start_time, end_time, interval):
    # Filter data for the specific day and time interval for both towers
    tower_data1_day = towerA_data[towerA_data['Date'] == day]
    tower_data2_day = towerB_data[towerB_data['Date'] == day]

    tower_data1_interval = tower_data1_day[(tower_data1_day['Time'] >= start_time) & (tower_data1_day['Time'] <= end_time)]
    tower_data2_interval = tower_data2_day[(tower_data2_day['Time'] >= start_time) & (tower_data2_day['Time'] <= end_time)]

    # Group data by specified time interval and calculate average temperature values
    tower1_avg_temp = tower_data1_interval.groupby(pd.Grouper(key='DateTime', freq=interval))[sensor_metric].mean()
    tower2_avg_temp = tower_data2_interval.groupby(pd.Grouper(key='DateTime', freq=interval))[sensor_metric].mean()

    # Convert average temperature values to numpy arrays
    tower1_avg_temp_np = tower1_avg_temp.values.reshape(-1, 1)
    tower2_avg_temp_np = tower2_avg_temp.values.reshape(-1, 1)

    # Calculate cosine similarity
    cosine_sim = cosine_similarity(tower1_avg_temp_np, tower2_avg_temp_np)

    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(tower1_avg_temp_np, tower2_avg_temp_np))

    return cosine_sim, rmse

In [5]:
tower_33_tar_file_path = "/mydata/actuallyuse/jer33-2018-11.tar.gz"
tower_34_tar_file_path = "/mydata/actuallyuse/jer34-2018-11.tar.gz"
tower_35_tar_file_path = "/mydata/actuallyuse/jer35-2018-11.tar.gz"

# Extracting the contents of relevant txt files for the towers
tower33Data = extractData(tower_33_tar_file_path)
tower34Data = extractData(tower_34_tar_file_path)
tower35Data = extractData(tower_35_tar_file_path)


tower33Data_processed = preprocess_data(tower33Data)
tower34Data_processed = preprocess_data(tower34Data)
tower35Data_processed = preprocess_data(tower35Data)


# Define the file paths for each tower



/tmp/ipykernel_741893/3083767795.py:63: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_tower_data = pd.concat([all_tower_data, tower_data], ignore_index=True)


File Names:  ['jer33/20181110/20181110_1600_JER0033_logger.txt.gz', 'jer33/20181110/20181110_0600_JER0033_logger.txt.gz', 'jer33/20181110/20181110_2200_JER0033_logger.txt.gz', 'jer33/20181110/20181110_0200_JER0033_logger.txt.gz', 'jer33/20181110/20181110_1200_JER0033_logger.txt.gz', 'jer33/20181110/20181110_0900_JER0033_logger.txt.gz', 'jer33/20181110/20181110_1900_JER0033_logger.txt.gz', 'jer33/20181110/20181110_2100_JER0033_logger.txt.gz', 'jer33/20181110/20181110_0500_JER0033_logger.txt.gz', 'jer33/20181110/20181110_1500_JER0033_logger.txt.gz', 'jer33/20181110/20181110_1100_JER0033_logger.txt.gz', 'jer33/20181110/20181110_0100_JER0033_logger.txt.gz', 'jer33/20181110/20181110_0700_JER0033_logger.txt.gz', 'jer33/20181110/20181110_2300_JER0033_logger.txt.gz', 'jer33/20181110/20181110_1700_JER0033_logger.txt.gz', 'jer33/20181110/20181110_1300_JER0033_logger.txt.gz', 'jer33/20181110/20181110_0300_JER0033_logger.txt.gz', 'jer33/20181110/20181110_1400_JER0033_logger.txt.gz', 'jer33/2018111

/tmp/ipykernel_741893/3083767795.py:63: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_tower_data = pd.concat([all_tower_data, tower_data], ignore_index=True)


File Names:  ['jer34/20181104/20181104_1700_JER0034_logger.txt.gz', 'jer34/20181104/20181104_2300_JER0034_logger.txt.gz', 'jer34/20181104/20181104_0700_JER0034_logger.txt.gz', 'jer34/20181104/20181104_0300_JER0034_logger.txt.gz', 'jer34/20181104/20181104_1300_JER0034_logger.txt.gz', 'jer34/20181104/20181104_0400_JER0034_logger.txt.gz', 'jer34/20181104/20181104_2000_JER0034_logger.txt.gz', 'jer34/20181104/20181104_1400_JER0034_logger.txt.gz', 'jer34/20181104/20181104_1000_JER0034_logger.txt.gz', 'jer34/20181104/20181104_0000_JER0034_logger.txt.gz', 'jer34/20181104/20181104_0800_JER0034_logger.txt.gz', 'jer34/20181104/20181104_1800_JER0034_logger.txt.gz', 'jer34/20181104/20181104_2200_JER0034_logger.txt.gz', 'jer34/20181104/20181104_0600_JER0034_logger.txt.gz', 'jer34/20181104/20181104_1600_JER0034_logger.txt.gz', 'jer34/20181104/20181104_1200_JER0034_logger.txt.gz', 'jer34/20181104/20181104_0200_JER0034_logger.txt.gz', 'jer34/20181104/20181104_1900_JER0034_logger.txt.gz', 'jer34/2018110

/tmp/ipykernel_741893/3083767795.py:63: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_tower_data = pd.concat([all_tower_data, tower_data], ignore_index=True)


File Names:  ['jer35/20181130/20181130_1700_JER0035_logger.txt.gz', 'jer35/20181130/20181130_0700_JER0035_logger.txt.gz', 'jer35/20181130/20181130_2300_JER0035_logger.txt.gz', 'jer35/20181130/20181130_0300_JER0035_logger.txt.gz', 'jer35/20181130/20181130_1300_JER0035_logger.txt.gz', 'jer35/20181130/20181130_2000_JER0035_logger.txt.gz', 'jer35/20181130/20181130_0400_JER0035_logger.txt.gz', 'jer35/20181130/20181130_1400_JER0035_logger.txt.gz', 'jer35/20181130/20181130_1000_JER0035_logger.txt.gz', 'jer35/20181130/20181130_0000_JER0035_logger.txt.gz', 'jer35/20181130/20181130_0800_JER0035_logger.txt.gz', 'jer35/20181130/20181130_1800_JER0035_logger.txt.gz', 'jer35/20181130/20181130_0600_JER0035_logger.txt.gz', 'jer35/20181130/20181130_2200_JER0035_logger.txt.gz', 'jer35/20181130/20181130_1600_JER0035_logger.txt.gz', 'jer35/20181130/20181130_1200_JER0035_logger.txt.gz', 'jer35/20181130/20181130_0200_JER0035_logger.txt.gz', 'jer35/20181130/20181130_1900_JER0035_logger.txt.gz', 'jer35/2018113

FileNotFoundError: [Errno 2] No such file or directory: '/mydata/actuallyuse/jer2-2018-11.tar.gz'

In [6]:
tower_2_tar_file_path = "/mydata/actuallyuse/jer02-2018-11.tar.gz"
tower_3_tar_file_path = "/mydata/actuallyuse/jer03-2018-11.tar.gz"
tower_4_tar_file_path = "/mydata/actuallyuse/jer04-2018-11.tar.gz"
tower_5_tar_file_path = "/mydata/actuallyuse/jer05-2018-11.tar.gz"
tower_6_tar_file_path = "/mydata/actuallyuse/jer06-2018-11.tar.gz"
tower_7_tar_file_path = "/mydata/actuallyuse/jer07-2018-11.tar.gz"
tower_8_tar_file_path = "/mydata/actuallyuse/jer08-2018-11.tar.gz"
tower_9_tar_file_path = "/mydata/actuallyuse/jer09-2018-11.tar.gz"
tower_10_tar_file_path = "/mydata/actuallyuse/jer10-2018-11.tar.gz"
tower_11_tar_file_path = "/mydata/actuallyuse/jer11-2018-11.tar.gz"
tower_12_tar_file_path = "/mydata/actuallyuse/jer12-2018-11.tar.gz"

# Extracting the contents of relevant txt files for the towers
tower2Data = extractData(tower_2_tar_file_path)
tower3Data = extractData(tower_3_tar_file_path)
tower4Data = extractData(tower_4_tar_file_path)
tower5Data = extractData(tower_5_tar_file_path)
tower6Data = extractData(tower_6_tar_file_path)
tower7Data = extractData(tower_7_tar_file_path)
tower8Data = extractData(tower_8_tar_file_path)
tower9Data = extractData(tower_9_tar_file_path)
tower10Data = extractData(tower_10_tar_file_path)
tower11Data = extractData(tower_11_tar_file_path)
tower12Data = extractData(tower_12_tar_file_path)

# Preprocess the data for each tower
tower2Data_processed = preprocess_data(tower2Data)
tower3Data_processed = preprocess_data(tower3Data)
tower4Data_processed = preprocess_data(tower4Data)
tower5Data_processed = preprocess_data(tower5Data)
tower6Data_processed = preprocess_data(tower6Data)
tower7Data_processed = preprocess_data(tower7Data)
tower8Data_processed = preprocess_data(tower8Data)
tower9Data_processed = preprocess_data(tower9Data)
tower10Data_processed = preprocess_data(tower10Data)
tower11Data_processed = preprocess_data(tower11Data)
tower12Data_processed = preprocess_data(tower12Data)

/tmp/ipykernel_741893/3083767795.py:63: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_tower_data = pd.concat([all_tower_data, tower_data], ignore_index=True)


File Names:  ['jer02/20181126/20181126_1600_JER0002_logger.txt.gz', 'jer02/20181126/20181126_0600_JER0002_logger.txt.gz', 'jer02/20181126/20181126_2200_JER0002_logger.txt.gz', 'jer02/20181126/20181126_0200_JER0002_logger.txt.gz', 'jer02/20181126/20181126_1200_JER0002_logger.txt.gz', 'jer02/20181126/20181126_0900_JER0002_logger.txt.gz', 'jer02/20181126/20181126_1900_JER0002_logger.txt.gz', 'jer02/20181126/20181126_2100_JER0002_logger.txt.gz', 'jer02/20181126/20181126_0500_JER0002_logger.txt.gz', 'jer02/20181126/20181126_1500_JER0002_logger.txt.gz', 'jer02/20181126/20181126_1100_JER0002_logger.txt.gz', 'jer02/20181126/20181126_0100_JER0002_logger.txt.gz', 'jer02/20181126/20181126_0700_JER0002_logger.txt.gz', 'jer02/20181126/20181126_2300_JER0002_logger.txt.gz', 'jer02/20181126/20181126_1700_JER0002_logger.txt.gz', 'jer02/20181126/20181126_1300_JER0002_logger.txt.gz', 'jer02/20181126/20181126_0300_JER0002_logger.txt.gz', 'jer02/20181126/20181126_1400_JER0002_logger.txt.gz', 'jer02/2018112

/tmp/ipykernel_741893/3083767795.py:63: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_tower_data = pd.concat([all_tower_data, tower_data], ignore_index=True)


ParserError: Error tokenizing data. C error: Expected 14 fields in line 2395, saw 17


In [6]:
tower33Data_processed.to_csv('/mydata/actuallyuse/towerdataset/tower33Data_processed.csv', index=False)
tower34Data_processed.to_csv('/mydata/actuallyuse/towerdataset/tower34Data_processed.csv', index=False)
tower35Data_processed.to_csv('/mydata/actuallyuse/towerdataset/tower35Data_processed.csv', index=False)
tower2Data_processed.to_csv('/mydata/actuallyuse/towerdataset/tower2Data_processed.csv', index=False)
tower3Data_processed.to_csv('/mydata/actuallyuse/towerdataset/tower3Data_processed.csv', index=False)
tower4Data_processed.to_csv('/mydata/actuallyuse/towerdataset/tower4Data_processed.csv', index=False)
tower5Data_processed.to_csv('/mydata/actuallyuse/towerdataset/tower5Data_processed.csv', index=False)
tower6Data_processed.to_csv('/mydata/actuallyuse/towerdataset/tower6Data_processed.csv', index=False)
tower7Data_processed.to_csv('/mydata/actuallyuse/towerdataset/tower7Data_processed.csv', index=False)
tower8Data_processed.to_csv('/mydata/actuallyuse/towerdataset/tower8Data_processed.csv', index=False)
tower9Data_processed.to_csv('/mydata/actuallyuse/towerdataset/tower9Data_processed.csv', index=False)
tower10Data_processed.to_csv('/mydata/actuallyuse/towerdataset/tower10Data_processed.csv', index=False)
tower11Data_processed.to_csv('/mydata/actuallyuse/towerdataset/tower11Data_processed.csv', index=False)
tower12Data_processed.to_csv('/mydata/actuallyuse/towerdataset/tower12Data_processed.csv', index=False)
tower33Data_processed.to_csv('/mydata/actuallyuse/towerdataset/tower33Data_processed.csv', index=False)
tower34Data_processed.to_csv('/mydata/actuallyuse/towerdataset/tower34Data_processed.csv', index=False)
tower35Data_processed.to_csv('/mydata/actuallyuse/towerdataset/tower35Data_processed.csv', index=False)

